In [134]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from textblob import TextBlob

In [135]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [136]:
df = pd.read_sql_query("SELECT * FROM spark_cleaned", con)

In [137]:
df.head(5)

,index,screen_name,date_time,text,retweeted,lang
0,0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0.0,en
1,1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0.0,en
2,2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0.0,en
3,3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0.0,en
4,4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0.0,en


In [138]:
df_drop_index = df.drop(columns = 'index')

In [139]:
df_drop_index.head()

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0.0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0.0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0.0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0.0,en
4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0.0,en


In [140]:
len(df_drop_index)

788

In [141]:
# remove duplicates 
df_no_dupes = df_drop_index.drop_duplicates()
df_no_dupes
len(df_no_dupes)

461

In [142]:
# use regex to remove @usernames

df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
#re.sub('@[^\s]+','',Tweet)

<ipython-input-142-59960d4ecb67>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
<ipython-input-142-59960d4ecb67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')


In [143]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup - I’m the crazy crypto/Tesla guy..,0.0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I don't believe I ever claimed Tesla had ...,0.0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media: “TESLA CRASHES INTO BUILDING WHILE ON ...,0.0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers ?,0.0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks. Now I got to add Tesla bot 🤔 So...,0.0,en
...,...,...,...,...,...
494,GlitchComputer,2021-09-05 22:57:15.000000,So how much did you get Wren? I just got my m...,NaN,en
495,DailyMail,2021-09-05 23:37:09.000000,"Tesla driver,43, kills his passenger and Flori...",NaN,en
496,Wilphred,2021-09-05 23:37:06.000000,Answer the question: do you support Texas' a...,NaN,en
497,JeffTutorials,2021-09-05 23:37:03.000000,"Just making beats in my Tesla, wbu? https://t....",NaN,en


In [144]:
# use regex to remove punctuation and emojis 
df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')

<ipython-input-144-0fde4d1c2c14>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')
<ipython-input-144-0fde4d1c2c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')


In [145]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup Im the crazy cryptoTesla guy,0.0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I dont believe I ever claimed Tesla had g...,0.0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media TESLA CRASHES INTO BUILDING WHILE ON AU...,0.0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers,0.0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks Now I got to add Tesla bot So t...,0.0,en
...,...,...,...,...,...
494,GlitchComputer,2021-09-05 22:57:15.000000,So how much did you get Wren I just got my mo...,NaN,en
495,DailyMail,2021-09-05 23:37:09.000000,Tesla driver43 kills his passenger and Florida...,NaN,en
496,Wilphred,2021-09-05 23:37:06.000000,Answer the question do you support Texas ant...,NaN,en
497,JeffTutorials,2021-09-05 23:37:03.000000,Just making beats in my Tesla wbu httpstcoogbu...,NaN,en


In [146]:
# make all text lowercase

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())

<ipython-input-146-4aad2dac7cc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: x.lower())


In [147]:
# remove stop words 
stop = stopwords.words('english')

df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

<ipython-input-147-e64ccaa07b0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [148]:
# stop
# list of stop words

In [149]:
df = df_no_dupes
df

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,yup im crazy cryptotesla guy,0.0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,dont believe ever claimed tesla great build qu...,0.0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,media tesla crashes building autopilot,0.0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,many followers,0.0,en
4,developrwannab,2021-09-04 13:28:53.000000,great thanks got add tesla bot thats kind deve...,0.0,en
...,...,...,...,...,...
494,GlitchComputer,2021-09-05 22:57:15.000000,much get wren got model 3 im terrified repairs...,NaN,en
495,DailyMail,2021-09-05 23:37:09.000000,tesla driver43 kills passenger florida woman 6...,NaN,en
496,Wilphred,2021-09-05 23:37:06.000000,answer question support texas antireproductive...,NaN,en
497,JeffTutorials,2021-09-05 23:37:03.000000,making beats tesla wbu httpstcoogbuakhprx,NaN,en


In [150]:
# sentiment analysis packages

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [151]:
# nlp 

df[['polarity', 'subjectivity']] = df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [152]:
df.head()

,screen_name,date_time,text,retweeted,lang,polarity,subjectivity
0,Zed18978371,2021-09-04 12:48:31.000000,yup im crazy cryptotesla guy,0.0,en,-0.600000,0.900000
1,TeslaOwls,2021-09-04 13:28:16.000000,dont believe ever claimed tesla great build qu...,0.0,en,0.427273,0.680909
2,vin_wiesel,2021-09-04 13:28:11.000000,media tesla crashes building autopilot,0.0,en,0.000000,0.000000
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,many followers,0.0,en,0.500000,0.500000
4,developrwannab,2021-09-04 13:28:53.000000,great thanks got add tesla bot thats kind deve...,0.0,en,0.525000,0.712500


In [153]:
# NLP for sentiment, neg, neu, pos, compound

for index, row in df['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    print(type(score))
    print(score)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
            # loc looks at curent row and if neg > pos, sentiment is negative
        df.loc[index, 'sentiment'] = 'negative'
    elif pos > neg:
        df.loc[index, 'sentiment'] = 'positive'
    else:
        df.loc[index, 'sentiment'] = 'neutral'
        
    df.loc[index, 'sentiment'] = '“neutral”'
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp
    

<class 'dict'>
{'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'compound': -0.34}
<class 'dict'>
{'neg': 0.175, 'neu': 0.585, 'pos': 0.24, 'compound': 0.296}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.9468}
<class 'dict'>
{'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'compound': 0.2023}
<class 'dict'>
{'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'compound': 0.2263}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'compound': 0.743}
<class 'dict'>
{'neg': 0.168, 'neu': 0.681, 'pos': 0.152, 'compound': -0.0772}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.242, 'neu': 0.529, 'pos': 0.229, 'compound': -0.0772}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.722, '

/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/sean/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.072, 'neu': 0.599, 'pos': 0.329, 'compound': 0.8225}
<class 'dict'>
{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.3182}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.104, 'neu': 0.808, 'pos': 0.088, 'compound': -0.128}
<class 'dict'>
{'neg': 0.065, 'neu': 0.514, 'pos': 0.421, 'compound': 0.8225}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'compound': 0.5859}
<class 'dict'>
{'neg': 0.0, 'neu': 0.707, 'pos': 0.293, 'compound': 0.6597}
<class 'dict'>
{'neg': 0.061, 'neu': 0.799, 'pos': 0.141, 'compound': 0.3612}
<class 'dict'>
{'neg': 0.0, 'neu': 0.154, 'pos': 0.846, 'compound': 0.6705}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.258, 'neu': 0.5, 'pos': 0.242, 'compound': -0.1027}
<class 'dict'>
{'neg': 0.149, '

<class 'dict'>
{'neg': 0.0, 'neu': 0.507, 'pos': 0.493, 'compound': 0.7783}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.3612}
<class 'dict'>
{'neg': 0.4, 'neu': 0.6, 'pos': 0.0, 'compound': -0.25}
<class 'dict'>
{'neg': 0.224, 'neu': 0.776, 'pos': 0.0, 'compound': -0.3818}
<class 'dict'>
{'neg': 0.155, 'neu': 0.709, 'pos': 0.136, 'compound': -0.0754}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'compound': 0.4767}
<class 'dict'>
{'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'compound': 0.3612}
<class 'dict'>
{'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'compound': -0.296}
<class 'dict'>
{'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.3182}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.054, 'neu': 0.659, 'pos': 0.287, 'compound': 0.7783}
<class 'dict'>
{'neg': 0.123, 'n

<class 'dict'>
{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'compound': 0.7003}
<class 'dict'>
{'neg': 0.1, 'neu': 0.699, 'pos': 0.201, 'compound': 0.5106}
<class 'dict'>
{'neg': 0.0, 'neu': 0.722, 'pos': 0.278, 'compound': 0.6124}
<class 'dict'>
{'neg': 0.0, 'neu': 0.548, 'pos': 0.452, 'compound': 0.5106}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'compound': 0.5106}
<class 'dict'>
{'neg': 0.153, 'neu': 0.847, 'pos': 0.0, 'compound': -0.5859}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.239, 'neu': 0.532, 'pos': 0.228, 'compound': -0.0377}
<class 'dict'>
{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.128}
<class 'dict'>
{'neg': 0.078, 'neu': 0.577, 'pos': 0.345, 'compound': 0.8495}
<class 'dict'>
{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.4939}
<class 'dict'>
{'neg': 0.0, 'n

<class 'dict'>
{'neg': 0.38, 'neu': 0.451, 'pos': 0.169, 'compound': -0.4344}
<class 'dict'>
{'neg': 0.0, 'neu': 0.161, 'pos': 0.839, 'compound': 0.6369}
<class 'dict'>
{'neg': 0.336, 'neu': 0.398, 'pos': 0.265, 'compound': -0.3818}
<class 'dict'>
{'neg': 0.242, 'neu': 0.565, 'pos': 0.194, 'compound': -0.2944}
<class 'dict'>
{'neg': 0.066, 'neu': 0.749, 'pos': 0.185, 'compound': 0.5719}
<class 'dict'>
{'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compound': 0.8849}
<class 'dict'>
{'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'compound': -0.3818}
<class 'dict'>
{'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.6712}
<class 'dict'>
{'neg': 0.243, 'neu': 0.485, 'pos': 0.272, 'compound': 0.0772}
<class 'dict'>
{'neg': 0.366, 'neu': 0.634, 'pos': 0.0, 'compound': -0.6369}
<class 'dict'>
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
<class 'dict'>
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4404}
<class 'dict'>
{'neg': 0.0, 'neu': 0.596, 'pos': 0.404, 'compound': 0.936}
<class 'dict'>

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 0 to 498
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   screen_name   461 non-null    object 
 1   date_time     461 non-null    object 
 2   text          461 non-null    object 
 3   retweeted     81 non-null     float64
 4   lang          461 non-null    object 
 5   polarity      461 non-null    float64
 6   subjectivity  461 non-null    float64
 7   sentiment     461 non-null    object 
 8   neg           461 non-null    float64
 9   neu           461 non-null    float64
 10  pos           461 non-null    float64
 11  compound      461 non-null    float64
dtypes: float64(7), object(5)
memory usage: 63.0+ KB
